In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from collections import Counter

# Load dataset
df = pd.read_csv("/kaggle/input/poems-dataset/poems-100.csv")
text = " ".join(df["text"].astype(str).tolist()).lower()
words = text.split()

# Vocabulary creation
vocab = list(set(words))
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# One-Hot Encoding Function
def one_hot_encode(sequence, vocab_size):
    encoded = np.zeros((len(sequence), vocab_size), dtype=np.float32)
    for i, idx in enumerate(sequence):
        encoded[i, idx] = 1.0
    return encoded

# Dataset Preparation
class PoetryDataset(Dataset):
    def __init__(self, words, word_to_idx, context_size=5):
        self.context_size = context_size
        self.data = []
        for i in range(len(words) - context_size):
            self.data.append((words[i:i+context_size], words[i+context_size]))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        context, target = self.data[index]
        context_idxs = [word_to_idx[word] for word in context]
        target_idx = word_to_idx[target]
        return torch.tensor(context_idxs, dtype=torch.long), torch.tensor(target_idx, dtype=torch.long)

# Model Definition
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

# Training Function
def train_model(model, dataset, epochs=10, lr=0.01):
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        total_loss = 0
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

# Prepare dataset
dataset = PoetryDataset(words, word_to_idx)

# Train One-Hot Model
vocab_size = len(vocab)
embedding_dim = 50
hidden_dim = 128
model = RNNModel(vocab_size, embedding_dim, hidden_dim)
train_model(model, dataset)

Epoch 1, Loss: 8.596567235973797
Epoch 2, Loss: 8.267568258233705
Epoch 3, Loss: 7.751467957071005
Epoch 4, Loss: 7.3649318893699105
Epoch 5, Loss: 7.021360802002743
Epoch 6, Loss: 6.834417656507406
Epoch 7, Loss: 6.636956230634859
Epoch 8, Loss: 6.427139007412909
Epoch 9, Loss: 6.378462053363104
Epoch 10, Loss: 6.149745958125915
